In [1]:
#import the neccessary libraries 
import csv
import pandas as pd
import requests
import json
from pprint import pprint 

# code necessary incase you'd like to search for a specific artist and their top 5 songs

artist = 'term=polyphia'
media = 'media=music'
entity = 'entity=song'
limit = 'limit=5'

url = 'http://itunes.apple.com/search?' + artist + media + entity + limit

In [2]:
#list of countries we narrow down to sort through our data

countries = ['hk', 'nl', 'au', 'ca', 'fr', 'jp', 'gb', 'de', 'us' ]

In [3]:
#create a dataframe of the current top 5 songs in the US

#set up api url and output the response data

url = "https://rss.itunes.apple.com/api/v1/us/apple-music/top-songs/all/5/explicit.json"
response = requests.get(url).json()

#create lists that will take in the name, album, artist, genre, and rank of each song
song = []
album = []
artist = []
genre = []
rank = []

counter = 0

for i,t in enumerate (response["feed"]["results"]):
    name_1=(i,t["name"])
    album_1=(t["collectionName"])
    song.append(name_1[1])
    album.append(album_1)
    artist.append(t["artistName"])
    genre.append(t["genres"][0]["name"])
    rank.append(counter + 1)
    
    counter += 1

#use the data from above to create a dataframe of the top 5 songs
us = {
     "Name": song,
    "Album": album,
    "Artist": artist,
    "Genre" : genre,
    "Rank" : rank
}

us_df = pd.DataFrame(us)
us_df

,Name,Album,Artist,Genre,Rank
0,The Box,Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,1
1,Start wit Me (feat. Gunna),Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,2
2,Peta (feat. Meek Mill),Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,3
3,Tip Toe (feat. A Boogie wit da Hoodie),Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,4
4,Woah,Woah - Single,Lil Baby,Hip-Hop/Rap,5


In [4]:
#create a dataframe similar to above that takes in all the top 5 songs from each country on our list

song_list = []
artist_list = []
album_list = []
country_list = []
genre_list = []
rank = []

for country in countries:
    top_list = []
    music_url = f"https://rss.itunes.apple.com/api/v1/{country}/apple-music/top-songs/all/5/explicit.json"
    
    try:
        response = requests.get(music_url).json()
        top_list.append(response["feed"]["results"])
        country_df = pd.DataFrame(top_list)
        for x in range (0,5):
            rank.append(x+1)
            song_list.append(country_df[x][0]['name'])
            country_list.append(country)
            artist_list.append(country_df[x][0]['artistName'])
            album_list.append(country_df[x][0]["collectionName"])
            genre_list.append(country_df[x][0]["genres"][0]["name"])
        
    except:
        print(f"Can't find {country}. Skipping... ")
top5_df = pd.DataFrame({
    "Country" : country_list,
    "Name" : song_list,
    "Artist" : artist_list,
    "Album" : album_list,
    "Genre" : genre_list,
    "Rank" : rank
})

top5_df['Country'] = top5_df['Country'].replace({'hk': 'Hong Kong',
                           'nl': 'Netherlands',
                           'au': 'Australia',
                           'ca': 'Canada',
                           'fr': 'France',
                           'jp': 'Japan',
                           'gb': 'UK',
                           'de': 'Germany',
                           'us': 'US'
                          })
top5_df['Source'] = 'Apple'
top5_df.head()

,Country,Name,Artist,Album,Genre,Rank,Source
0,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,1,Apple
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,流行樂,2,Apple
2,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,國語流行樂,3,Apple
3,Hong Kong,Show Yourself,Idina Menzel & Evan Rachel Wood,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,4,Apple
4,Hong Kong,Memories,魔力紅樂團,Memories - Single,流行樂,5,Apple


In [5]:
from googletrans import Translator 
translator = Translator()

top5_df['Translated Genre'] = top5_df['Genre'].apply(translator.translate, dest='en').apply(getattr, args=('text',))
top5_df['Translated Genre'] = top5_df['Translated Genre'].replace({'pop music' : 'Pop', 
                                                                   'Mandarin pop music' : 'Pop',
                                                                   'Alternative Music' : 'Alternative',
                                                                   'R & B, soul' : 'R&B/Soul',
                                                                   'priest' : 'Priest'
                                                                })
top5_df.head()

,Country,Name,Artist,Album,Genre,Rank,Source,Translated Genre
0,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,1,Apple,Soundtrack
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,流行樂,2,Apple,Pop
2,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,國語流行樂,3,Apple,Pop
3,Hong Kong,Show Yourself,Idina Menzel & Evan Rachel Wood,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,4,Apple,Soundtrack
4,Hong Kong,Memories,魔力紅樂團,Memories - Single,流行樂,5,Apple,Pop


In [6]:
output_file = top5_df.to_csv('../Resources/appleAPI_data_new.csv', index = None, header=True)